In [1]:
# imports
from pathlib import Path
from time import time

import numpy as np
import pandas as pd

from utils import (
    read_single_problem_from_path_as_adjacency,
    read_single_problem_from_path_as_sparse,
    read_single_problem_from_path_as_sparse_from_adjacency,
    order_nodes_in_descending_order,
    check_validity_for_adjacency,
    transform_node_clique_to_zero_one,
    transform_zero_one_clique_to_nodes
)
from constants import BASE_INSTANCES_FILES, OTHER_INSTANCES_FILES, OTHER_INSTANCES_BEST_KNOWN
from heuristics import (
    descending_degree_glutonous_heuristic,
    dynamic_descending_degree_glutonous_heuristic,
    descending_degree_random_heuristic,
    multiple_descending_degree_random_heuristic
)

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DIR = INSTANCES_DIR / "other_instances"


In [2]:
_, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "random-10.col"
)
print(graph)
clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
print(clique)
print("Is the clique valid ? ", check_validity_for_adjacency(graph, clique))


[[0 1 0 1 1 1 0 0 1 0]
 [1 0 1 0 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0 0 1]
 [1 0 1 0 0 0 0 0 1 0]
 [1 1 0 0 0 1 1 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 0 1 1 0 1 1 1]
 [0 1 0 0 1 0 1 0 0 1]
 [1 1 0 1 0 1 1 0 0 0]
 [0 1 1 0 1 0 1 1 0 0]]
[0 1 0 0 1 0 1 1 0 1]
Is the clique valid ?  True


In [3]:
# Check that clique representation utils work
number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "random-10.col"
)
clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
print(clique)
print(transform_zero_one_clique_to_nodes(clique))
print(transform_node_clique_to_zero_one(number_of_nodes, transform_zero_one_clique_to_nodes(clique)))

[0 1 0 0 1 0 1 1 0 1]
[1, 4, 6, 7, 9]
[0 1 0 0 1 0 1 1 0 1]


In [4]:
# Build clique size and times of execution for our base glutonous heuristics
files = []
instance_reading_times = []
clique_creating_times = []
total_times = []
clique_sizes = []
methods = []

for file in BASE_INSTANCES_FILES:
    for method, verbose_method in [
        (descending_degree_glutonous_heuristic, "desc_base"),
        (dynamic_descending_degree_glutonous_heuristic, "desc_dyn"),
        (descending_degree_random_heuristic, "desc_ran"),
        (multiple_descending_degree_random_heuristic, "mult_ran")
    ]:
        # Method and file adding
        methods.append(verbose_method)
        files.append(file)

        # File loading
        start_time = time()
        _, _, graph, degrees = read_single_problem_from_path_as_adjacency(
            instance_path=BASE_INSTANCES_DIR / file
        )
        end_of_read_time = time()
        instance_reading_times.append(end_of_read_time - start_time)

        # Clique building
        clique = method(graph=graph, degrees=degrees)
        clique_sizes.append(np.sum(clique))
        end_of_clique_time = time()
        clique_creating_times.append(end_of_clique_time - end_of_read_time)

        assert check_validity_for_adjacency(graph, clique)
        total_times.append(time() - start_time)
    
    # At the end of each method, add visual separator
    for display_list in [files, instance_reading_times, clique_creating_times, total_times, clique_sizes, methods]:
        display_list.append("/////")

# Display for most basic heuristics

display_dataframe = pd.DataFrame(
    {
        "file": files,
        "method": methods,
        "clique size": clique_sizes,
        "instance time": instance_reading_times,
        "clique time": clique_creating_times,
        "total time": total_times,
    }
)
print(display_dataframe)


              file     method clique size instance time clique time total time
0   brock200_2.col  desc_base           7      0.051001       0.004   0.055001
1   brock200_2.col   desc_dyn           7      0.031995    0.004037   0.036033
2   brock200_2.col   desc_ran           8      0.025962       0.004   0.029962
3   brock200_2.col   mult_ran           8         0.026    0.416161    0.44216
4            /////      /////       /////         /////       /////      /////
5    dsjc125.1.col  desc_base           4      0.002998    0.001002      0.004
6    dsjc125.1.col   desc_dyn           4         0.003       0.001      0.004
7    dsjc125.1.col   desc_ran           3      0.004046    0.001976   0.006022
8    dsjc125.1.col   mult_ran           4      0.001977    0.197032   0.199009
9            /////      /////       /////         /////       /////      /////
10   random-10.col  desc_base           5           0.0         0.0        0.0
11   random-10.col   desc_dyn           5         0.

In [6]:
# Test on other graphs with known bounds.
files = []
instance_reading_times = []
total_times = []
clique_sizes = []
methods = []
best_knowns = []

for file in OTHER_INSTANCES_FILES:
    for method, verbose_method in [
        (descending_degree_glutonous_heuristic, "desc_base"),
        (dynamic_descending_degree_glutonous_heuristic, "desc_dyn"),
        (descending_degree_random_heuristic, "desc_ran"),
        (multiple_descending_degree_random_heuristic, "mult_ran")
    ]:
        # Method, file and best adding
        methods.append(verbose_method)
        files.append(file)
        best_knowns.append(OTHER_INSTANCES_BEST_KNOWN[file])

        # File loading
        start_time = time()
        _, _, graph, degrees = read_single_problem_from_path_as_adjacency(
            instance_path=OTHER_INSTANCES_DIR / file
        )
        end_of_read_time = time()
        instance_reading_times.append(end_of_read_time - start_time)

        # Clique building
        clique = method(graph=graph, degrees=degrees)
        clique_sizes.append(np.sum(clique))
        end_of_clique_time = time()

        assert check_validity_for_adjacency(graph, clique)
        total_times.append(time() - start_time)
    
    # At the end of each method, add visual separator
    for display_list in [files, instance_reading_times, total_times, clique_sizes, methods, best_knowns]:
        display_list.append("/////")

# Display
display_dataframe = pd.DataFrame(
    {
        "file": files,
        "method": methods,
        "clique size": clique_sizes,
        "best": best_knowns,
        "instance time": instance_reading_times,
        "total time": total_times,
    }
)
print(display_dataframe)

                  file     method clique size   best instance time total time
0       brock800_4.txt  desc_base          14     26      0.527011   0.550036
1       brock800_4.txt   desc_dyn          14     26      0.538964    0.57899
2       brock800_4.txt   desc_ran          16     26      0.601732   0.630384
3       brock800_4.txt   mult_ran          15     26      0.571287   5.056861
4                /////      /////       /////  /////         /////      /////
5          C2000.5.txt  desc_base          10     16      2.679512   2.729403
6          C2000.5.txt   desc_dyn          10     16      2.471838   2.548136
7          C2000.5.txt   desc_ran          11     16      2.468839    2.53051
8          C2000.5.txt   mult_ran          11     16      2.520718   7.849467
9                /////      /////       /////  /////         /////      /////
10          C500.9.txt  desc_base          43     57      0.279126   0.312553
11          C500.9.txt   desc_dyn          42     57      0.2942

In [ ]:
# Test different size of choices for the nodes for random heuristic
files = []
instance_reading_times = []
clique_creating_times = []
total_times = []
clique_sizes = []
methods = []

for file in ["brock200_2.col"]:
    for k in range(1, 11):
        # Method and file adding
        methods.append(f"multi_ran_{k}")
        files.append(file)

        # File loading
        start_time = time()
        _, _, graph, degrees = read_single_problem_from_path_as_adjacency(
            instance_path=BASE_INSTANCES_DIR / file
        )
        end_of_read_time = time()
        instance_reading_times.append(end_of_read_time - start_time)

        # Clique building
        clique = multiple_descending_degree_random_heuristic(graph=graph, degrees=degrees, size_of_choice=k, number_of_iterations=20)
        clique_sizes.append(np.sum(clique))
        end_of_clique_time = time()
        clique_creating_times.append(end_of_clique_time - end_of_read_time)

        assert check_validity_for_adjacency(graph, clique)
        total_times.append(time() - start_time)
    
    # At the end of each method, add visual separator
    for display_list in [files, instance_reading_times, clique_creating_times, total_times, clique_sizes, methods]:
        display_list.append("/////")


114.0 1 200
114.0 1 199
114.0 1 198
114.0 1 197
114.0 1 196
114.0 1 195
114.0 1 194
114.0 1 193
114.0 1 192
114.0 1 191
114.0 1 190
114.0 1 189
114.0 1 188
114.0 1 187
114.0 1 186
114.0 1 185
114.0 1 184
114.0 1 183
114.0 1 182
114.0 1 181
114.0 1 180
114.0 1 179
114.0 1 178
114.0 1 177
114.0 1 176
114.0 1 175
114.0 1 174
114.0 1 173
114.0 1 172
114.0 1 171
114.0 1 170
114.0 1 169
114.0 1 168
114.0 1 167
114.0 1 166
114.0 1 165
114.0 1 164
114.0 1 163
114.0 1 162
114.0 1 161
114.0 1 160
114.0 1 159
114.0 1 158
114.0 1 157
114.0 1 156
114.0 1 155
114.0 1 154
114.0 1 153
114.0 1 152
114.0 1 151
114.0 1 150
114.0 1 149
114.0 1 148
114.0 1 147
114.0 1 146
114.0 1 145
114.0 1 144
114.0 1 143
114.0 1 142
114.0 1 141
114.0 1 140
114.0 1 139
114.0 1 138
114.0 1 137
114.0 1 136
114.0 1 135
114.0 1 134
114.0 1 133
114.0 1 132
114.0 1 131
114.0 1 130
114.0 1 129
114.0 1 128
114.0 1 127
114.0 1 126
114.0 1 125
114.0 1 124
114.0 1 123
114.0 1 122
114.0 1 121
114.0 1 120
114.0 1 119
114.0 1 118
114.

In [ ]:
# Display for different size of choices for random heuristic

display_dataframe = pd.DataFrame(
    {
        "file": files,
        "method": methods,
        "clique size": clique_sizes,
        "instance time": instance_reading_times,
        "clique time": clique_creating_times,
        "total time": total_times,
    }
)
print(display_dataframe)

              file        method clique size instance time clique time  \
0   brock200_2.col   multi_ran_1           7      0.031824    0.154001   
1   brock200_2.col   multi_ran_2           7      0.029968    0.156307   
2   brock200_2.col   multi_ran_3           8      0.030007    0.140332   
3   brock200_2.col   multi_ran_4           8      0.029058    0.141943   
4   brock200_2.col   multi_ran_5           7         0.029    0.191049   
5   brock200_2.col   multi_ran_6           7      0.033997    0.155026   
6   brock200_2.col   multi_ran_7           8      0.032871    0.147592   
7   brock200_2.col   multi_ran_8           7      0.030056    0.139665   
8   brock200_2.col   multi_ran_9           8       0.02802    0.138627   
9   brock200_2.col  multi_ran_10           8      0.039962    0.178438   
10           /////         /////       /////         /////       /////   

   total time  
0    0.185826  
1    0.186275  
2    0.170339  
3    0.171001  
4    0.220049  
5    0.189023  